# W207 Final Project : Facial Keypoint Detection 
# Team: Joanie Weaver, Sandip Panesar, Jackie Nichols, Rakesh Walisheter
W207 Tuesday @4pm

ref: https://www.kaggle.com/c/facial-keypoints-detection

# Data Cleaning File

## Setup
Imports, reading in files, etc..

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
import zlib

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rc
from matplotlib.ticker import PercentFormatter
import pickle


In [ ]:
#Load the pickle files

train = pickle.load( open( "data/train.p", "rb" ) )
test = pickle.load( open("data/test.p", "rb"))

train.rename(columns = {'level_0' : 'index'}, inplace = True)

train_duplicates = pickle.load( open("data/traindup.p", "rb"))
test_duplicates = pickle.load( open("data/testdup.p", "rb"))

train_duplicates.set_index('index', inplace=True, drop=False)
print("Test shape is: ", test.shape)
print("Train shape is: ", train.shape)

print("Test duplicates shape is: ", test_duplicates.shape)
print("Train duplicates shape is: ", train_duplicates.shape)


In [ ]:
#HELPER FUNCTIONS to reset the train and test dataframes

def reset_train_df():
    #train = df['train'].reset_index().copy()
    new_train = train.reset_index().copy()
    #Get the images and perform a checksum on every image in train: https://www.geeksforgeeks.org/zlib-adler32-in-python/
    new_train['check_sum'] = train.image.map(lambda x: zlib.adler32(x))
    new_train.pop('level_0')
    return new_train

def reset_test_df():
    #test = df['test'].reset_index().copy()
    new_test = test.reset_index().copy()
    #Get the images and perform a checksum on every image in train: https://www.geeksforgeeks.org/zlib-adler32-in-python/
    new_test['check_sum'] = test.image.map(lambda x: zlib.adler32(x))
    new_test.pop('level_0')
    return new_test

def get_coordinate_columns():
    coordinates = [c for c in train.columns if c.endswith('_x') | c.endswith('_y')]
    return coordinates

In [ ]:
#Creating a copy of the train data in train_data in case you want to add columns back in from df[train]
train_data=train.copy(deep=True)
train_data

## Outlier Identification
As we saw in the EDA, there are a variety of types of images with a variety of keypoints. Below, we will remove some of the images we saw as outliers in the EDA.

Outlier types:
- Mislabelled images
- Weird/bad images
- All outliers (i.e. images that contain keypoints that are greater than 3 standard deviations away from the mean for that keypoint)

In [ ]:
#CODE CELL FOR JOANIE

#This block is identifying and counting all outliers
#Outliers are images that contain keypoints > 3std from mean
def find_outliers():
    train=train_data.drop(["image"],axis=1)
    described_train=train.describe().T
    std=described_train["std"]
    mean=described_train["mean"]
    q1=described_train["25%"]
    q3=described_train["75%"]
    iqr=q3-q1

    #If we define outliers using IQR
    #outlier_low=q1-1.5*iqr
    #outlier_high=q3+1.5*iqr

    #If we define outliers using std
    outlier_low=mean-3*std
    outlier_high=mean+3*std



    #Keep track of these images in a list
    outlier_images=[]
    outlier_dict={}

    #Iterate through the data to find outliers based on whether they are lower/higher than defined outlier boundaries
    for col in train.columns:
        indices=list(np.where((train[col] < outlier_low[col]) | (train[col] > outlier_high[col]))[0])
        outlier_images.extend(indices)
        for i in indices:
            temp=outlier_dict.get(i,[])
            temp.append(col[:-1])
            outlier_dict[i]=temp

    #Only count each index once
    outliers=np.unique(outlier_images)
    outliers
    print("Finding points 3 standard deviations away from the mean results in ",len(outliers),
        "images being classified as outliers")
    print("This represents",len(outliers)/train.shape[0]*100,"% of our total data")

    #print(train.shape)
    
    return outliers




In [ ]:
#CODE CELL FOR JOANIE

#This function is removing the worst outliers
#The worst outliers are the mislabelled images and the weird/bad images
def drop_worst_outliers():

    print("Before dropping worst outliers train shape is: ", train.shape)
    print("Before dropping worst outliers train duplicates shape is: ", train_duplicates.shape)
    miss_labelled = [1747, 1877, 1907,2199] #These are the images with keypoints that are not right
    bad_images = [6492,6493,2430,3697] #These are the two collages and the two cartoons

    worst_outliers = miss_labelled + bad_images

    #Drop with inplace drops inplace
    #train_data.drop(index=worst_outliers,inplace=True)
    train.drop(index=worst_outliers,inplace=True,errors='ignore')
    train_duplicates.drop(index=worst_outliers,inplace=True,errors='ignore')
    print("After dropping worst outliers train shape is: ", train.shape)
    print("After dropping worst outliers train duplicates shape is: ", train_duplicates.shape)

#This function is for removing all outliers as defined above
def drop_all_outliers():
    print("Before dropping all outliers train shape is: ", train.shape)
    print("Before dropping all outliers train duplicates shape is: ", train_duplicates.shape)
    
    outliers = find_outliers()
    overlap = [bad for bad in outliers if bad in train_duplicates.index]

    train.drop(index=outliers,inplace=True, errors='ignore')
    #Drop the overlap outliers and duplicates
    train_duplicates.drop(index=overlap,inplace=True,errors='ignore')
    print("After dropping all outliers train shape is: ", train.shape)
    print("After dropping all outliers train duplicates shape is: ", train_duplicates.shape)

def drop_overlap_outliers():
    print("Before dropping overlap outliers train shape is: ", train.shape)
    print("Before dropping overlap outliers train duplicates shape is: ", train_duplicates.shape)
    
    outliers = find_outliers()
    overlap = [bad for bad in outliers if bad in train_duplicates.index]

    print("There are", len( overlap), " images that are outliers that appear in train duplicates")
    
    train.drop(index=overlap,inplace=True, errors='ignore')
    #Drop the overlap outliers and duplicates
    train_duplicates.drop(index=overlap,inplace=True,errors='ignore')
    print("After dropping overlap outliers train shape is: ", train.shape)
    print("After dropping overlap outliers train duplicates shape is: ", train_duplicates.shape)




## Helper Code for Duplicate Data in Train and Test datasets

In [ ]:
#CODE CELL FOR JACKIE
# Remove duplicates in the train dataset by taking the mean of all values for that image in each label 
def remove_train_duplicates( train_duplicates,verbose=True):
    # First let's reset the index since we've been working on the df 
        
    train = reset_train_df()
    #train_duplicates.reset_index()

    #Get all of the coordinates
    coordinates = get_coordinate_columns()

    #Create an empty df with the coordinate columns in place
    final_images = train[(train.index == -1)][coordinates].copy()
    final_check_sum = train_duplicates.check_sum.unique()

    #For each unique check_sum in duplicates...
    for check_sum in train_duplicates.check_sum.unique():
        #Get all of the duplicates with the same check_sum
        duplicates = train_duplicates[(train_duplicates.check_sum == check_sum)]['index'].values
        
        #Get the first image that appears in the train dataset with this check_sum
        image = train[(train['index'].isin(duplicates))].image.values[0]
        #Take the mean of all the coordinate columns - this is what we will use for the final single image
        fixed = pd.DataFrame(pd.DataFrame(train[(train['index'].isin(duplicates))], columns=coordinates).mean(axis = 0)).T
        #Make sure to include the actual image (lol)
        fixed['image'] = [image]
        fixed['check_sum'] = check_sum
        fixed['index'] = duplicates[0] #take first index
        
        #Append it to the list of final_images
        final_images = final_images.append(fixed, ignore_index = True)
        
            
    #For reporting purposes: 
    if verbose: print("="*13 + "Train" + "="*13)
    if verbose: print("Before delete:     %s" % str(train.shape))

    #Remove the duplicates from train - danger, danger, must replace them
    train = train[~(train['index'].isin(train_duplicates['index'].values))]
    if verbose: print("After  delete:     %s" % str(train.shape))

    #Dump the final images that were duplicates so we can take a look at them after the processing. 
    pickle.dump(final_check_sum, open( "data/final_check_sum.p", "wb" ) )
 

    #Replace removed duplicates with final_images
    train = train.append(final_images, ignore_index = True).reset_index()
    train['check_sum'] = train.image.map(lambda x: zlib.adler32(x))
    #train.drop(columns=['index'], inplace = True)
    if verbose: print("After  append:     %s" % str(train.shape))
    return train



In [ ]:
#CODE CELL FOR JACKIE
##########Test Data set

#Now do the same for test, this will be easier since we don't need
#to deal with points and taking the mean
def remove_test_duplicates(verbose=True):
#We can do this differently since we don't need to take the mean. 
#Go through the test and only add items to the final test image if
#we do not already have the check_sum. If we find the check_sum, don't
#add it it's a duplicate. 
    test = reset_test_df()
    if verbose: print("="*13 + "Test=" + "="*13)
    if verbose: print("Before delete:     %s" % str(test.shape))
    test = reset_test_df()
    #Create an empty df with the coordinate columns in place
    final_test_images = test[(test.index == -1)]
    
    for test_index, check_sum in zip(test['index'], test.check_sum):
        if not (check_sum in list(final_test_images.check_sum.values)):
            final_test_images = final_test_images.append(test.loc[(test['index'] == test_index)], ignore_index = True)
    
    if verbose: print("After  delete:     %s" % str(final_test_images.shape))
    return final_test_images


## Remove worst outliers

In [ ]:
#CODE CELL FOR JOANIE
#only drop the worst outliers for now
#
drop_overlap_outliers()

drop_worst_outliers()

## Remove duplicates

In [ ]:
#CODE CELL FOR JACKIE

print("Applying EDA fix for duplicates")
print()
train = remove_train_duplicates(train_duplicates)
print()
test = remove_test_duplicates()



## Save Clean Data to a Pickle file

In [ ]:
#CODE CELL FOR JACKIE
print(train.columns)
print(test.shape)
#Pickle train and test so that we can jump in with cleaning this data
pickle.dump( train, open( "data/clean_train.p", "wb" ) )
pickle.dump(test, open( "data/clean_test.p", "wb" ))


In [ ]:

#Load the fixed images from the duplication process
fixed_images = pickle.load( open( "data/final_check_sum.p", "rb" ) )

In [ ]:
#Print clean images:

def show_fixed_images(fixed_images):
    ## TRAIN 
    # Let's view some of these duplicated train images
    fig = plt.figure(figsize=(20,20))
    fig.suptitle('Sample of Cleaned Duplicate images from the Train dataset\n n= 50', size = 20,  y = 1.04, weight = 'bold')
    #Get the point coordinates for example: mouth_center_top_lip_x
    coordinates = get_coordinate_columns()
    #print(coordinates)
    
   
    match_pts = pd.DataFrame(columns =['Points_Found', 'Count'])

     #Loop through and plot each of the 50 images.  
    for i, check_sum in enumerate (fixed_images):
        if i == 50:
            break
        plt.subplot(10,5,i+1)
        img = train[(train['check_sum'] == check_sum)].image.values[0].reshape(96,96)
        #These are the points that have been identified on the images
        points = train[(train['check_sum'] == check_sum)][coordinates].values[0]
        idx = train[(train['check_sum'] == check_sum)]['index']
        plt.imshow(img, cmap = 'gray')
        plt.axis('off')
        matching_pts = 0

        for pts in range(0, 30, 2):
            x_point, y_point = (points[pts], points[pts+1])
            if not (np.isnan(x_point)) and not (np.isnan(y_point)):
                matching_pts += 1
                #Add the point to the plot
                plt.plot(x_point, y_point, 'o', color = "red", markersize = 5)
                

        plt.title("Image #:[%d]\n#Points:[%d]" % (idx, matching_pts))
        if matching_pts in match_pts["Points_Found"].values:
                match_pts.loc[match_pts['Points_Found'] == matching_pts, 'Count'] = match_pts.loc[match_pts['Points_Found'] == matching_pts, 'Count'] + 1
        else:
            match_pts = match_pts.append({'Points_Found':matching_pts,'Count': 1},ignore_index=True)


    #We should save every image with a marker so we can look at them
    #im = Image.fromarray(img)
    #im.save(file_name)

    plt.tight_layout()
    plt.show()





In [ ]:
show_fixed_images(fixed_images)